In [ ]:
#Sample Lucy Bot for one of the Use Cases we tried. This code is for helping users to buy products online from Amazon


#importing Python Libraries
import logging
from flask import Flask
from flask_assistant import Assistant, ask, tell, context_manager
import os
import pandas as pd
from amazon.api import AmazonAPI
import bottlenose.api
import pandas as pd

#API.AI Credentials
os.environ['CLIENT_ACCESS_TOKEN'] = 'Your Client Access Token from API.AI'
os.environ['DEV_ACCESS_TOKEN'] = 'Your Dev Access Token from API.AI'

app = Flask(__name__)
assist = Assistant(app,'/the_bot')
logging.getLogger('flask_assistant').setLevel(logging.DEBUG)


#Calling API.AI Entities
#Sample Entity : greetings to greet the user when it starts interacting with Lucy
@assist.action('greetings')
def greetings():
    speech = """Hi I am Lucy! I can help you with ordering XYZ (You can enter any Manufacturer's name) products. Would you like to buy some XYZ Products today?"""
    context_manager.add('XYZ', lifespan=1)
    return ask(speech)

#Sample Entity: confirm to take actions based on user's reply
@assist.action('confirm')
def confirm_flow(answer):
    if 'no' in answer:
        speech = "Ok sounds good. Let's talk some other time?"
        return ask(speech)
    else:
        speech = "Ok sounds good. Which product would you like to buy today?"
        context_manager.add('product')
        return ask(speech)

#Sample Entity: product_search to send Product Details(from API.AI's json file to Amazon API)
@assist.action('product_search', mapping={'Brand': 'Brand','ProductDetails': 'ProductDetails','HairType':'HairType',
                                          'ProductSize': 'ProductSize'})
def product_search(Brand,ProductDetails):
    amazon_CA= AmazonAPI('Your AMAZON_ACCESS_KEY', 'Your AMAZON_SECRET_KEY', 'Your AMAZON_ASSOC_TAG')
    products = amazon_CA.search(Keywords=Brand+' '+ProductDetails, SearchIndex='All')
    series =[]
    for i, product in enumerate(products):
        series.append(product.title)
    speech = 'I will order' + series[0] + 'for you'
    #context_manager.add('delivery-info', lifespan=10)
    #context_manager.set('delivery-info', 'address', address)
    return ask(speech)


if __name__ == '__main__':
    app.run(debug=False)